In [4]:
import pandas as pd
from libraries.utility import Utility as mutil
from libraries.db_base import db_base
import os
import time

from libraries.utility import Utility as mutil
from libraries.db_base import db_base
from libraries.db_ins_fake_data import fake_data_to_db
from libraries.custom_excel_output import custom_excel_output
import pandas as pd
import pandasql as psql
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import warnings
import time
import os
import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))


from pyspark.sql import SparkSession



In [5]:
def create_spark_sesssion():
    # Create SparkSession
    spark = SparkSession.builder \
        .appName("MasterSession") \
        .config("spark.jars", "/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar") \
        .config("spark.jars", "/usr/share/java/postgresql.jar") \
        .config("spark.jars", "/usr/share/java/mysql-connector-j-8.2.0.jar") \
        .getOrCreate()    
    return spark

def test_sql_server_connection(query,spark):
    
    #Location of database drivers
    #postgres
    #/usr/share/java/postgresql.jar
    #mysql
    #/usr/share/java/mysql-connector-j-8.2.0.jar
    #mssql
    #/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar

    # Start the timer
    start_time = time.time()
    


    # Database connection properties
    database_url = "jdbc:sqlserver://mssql1:1433;databaseName=products"
    database_properties = {
        "user": "sa",
        "password": "Python2028",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }

    
    # Read data from MSSQL
    #df = spark.read.jdbc(url=database_url, table="products", properties=database_properties)

    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)

    # Show the data
    #df.show()
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    return df


def test_postgress_sql(query,spark):
  
    #Location of database drivers
    #postgres
    #/usr/share/java/postgresql.jar
    #mysql
    #/usr/share/java/mysql-connector-j-8.2.0.jar
    #mssql
    #/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar

    # Start the timer
    start_time = time.time()
    

    # Database connection properties
    database_url = "jdbc:postgresql://postsql1:5432/finance"
    database_properties = {
        "user": "postgres",
        "password": "Python2028",
        "driver": "org.postgresql.Driver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }
    
    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)

    # Show the data
    #df.show()
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    return df

def test_mysql_server(query,spark):
    
    #Location of database drivers
    #postgres
    #/usr/share/java/postgresql.jar
    #mysql
    #/usr/share/java/mysql-connector-j-8.2.0.jar
    #mssql
    #/usr/share/java/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre11.jar

    # Start the timer
    start_time = time.time()
    

    # Database connection properties
    database_url = "jdbc:mysql://mysql1:3306/customers"
    database_properties = {
        "user": "root",
        "password": "Python2028",
        "driver": "com.mysql.cj.jdbc.Driver",
        "encrypt": "true",
        "trustServerCertificate": "true"  # Add this line        
    }
    
    # Read data from MSSQL
    df = spark.read.jdbc(url=database_url, table=query, properties=database_properties)
    
    # Stop the timer and calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    print(f"Time elapsed: {minutes} minutes and {seconds} seconds")
    
    return df
    


In [6]:

u = mutil()
# Start the timer
start_time = time.time()

# Initialize a SparkSession
spark = create_spark_sesssion()


dbprod = db_base("products")
get_product_info_sql=dbprod.get_sql_query_from_query_key("get_product_info")

query= "(" + get_product_info_sql + ") AS query_table"

prod_info=test_sql_server_connection(query=query,spark=spark)


dbfin = db_base("finance",svr_type='postsql')

get_account_activity = dbfin.get_sql_query_from_query_key("get_account_activity")

query= "(" + get_account_activity + ") AS query_table"

fin_account_activity=test_postgress_sql(query,spark)


dbcust = db_base("customers",svr_type='mysql')


query = dbcust.get_sql_query_from_query_key("get_customer_product_history1")

query = "(" + query + ") AS query_table"

cust_products = test_mysql_server(query,spark)




# Assuming cust_products, prod_info, and fin_account_activity are already loaded as DataFrames
# For example, they can be loaded from a source like this:
# cust_products = spark.read.format("source_format").load("path_to_cust_products")
# prod_info = spark.read.format("source_format").load("path_to_prod_info")
# fin_account_activity = spark.read.format("source_format").load("path_to_fin_account_activity")

# Register DataFrames as temporary views
cust_products.createOrReplaceTempView("cust_products")
prod_info.createOrReplaceTempView("prod_info")
fin_account_activity.createOrReplaceTempView("fin_account_activity")


# SQL query
sql_query = '''
SELECT
    DISTINCT
    cp.id AS customer_id,
    cp.f_name,
    cp.l_name,
    p.product_name,
    p.product_type,
    fa.amt_usd,
    fa.post_date,
    fa.channel_desc,
    fa.partner_desc,
    fa.location_name,
    fa.account_name,
    fa.account_type
FROM cust_products cp
JOIN prod_info p ON p.id = cp.product_id
JOIN fin_account_activity fa ON fa.customer_id = cp.id AND fa.product_id = cp.product_id
'''

# Execute SQL query
result_df = spark.sql(sql_query)

# Show the result
result_df.show()

# Stop the SparkSession when done
spark.stop()


# Stop the timer and calculate elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

# Convert elapsed time to minutes and seconds
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"Overall Time: {minutes} minutes and {seconds} seconds")




('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found (0) (SQLDriverConnect)")
Database : products, Connection Good: False
Time elapsed: 0 minutes and 0 seconds
connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

Database : finance, Connection Good: False
Time elapsed: 0 minutes and 0 seconds
2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)
Database : customers, Connection Good: False
Time elapsed: 0 minutes and 0 seconds
+-----------+--------------------+--------------------+--------------------+--------------------+-------+-------------------+--------------------+--------------------+--------------+--------------------+------------+
|customer_id|              f_name|              l_name|        product_name|        product_type|amt_usd|          post_date|        channel_desc|        partner_desc| location_name|      